- 실습중인 쇼핑몰 살품은 여러페이지에 걸쳐 진열되어 있음
    - 함수로 구성 후 여러페이지 크롤링

In [9]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import ssl
context = ssl._create_unverified_context()
url="http://jolse.com/category/toners-mists/1019/"

### 함수를 기능별로 세분화
- 요청 및 파싱 객체 생성 : get_request_product(url), 반환값-bs4 객체
- 전체 정보 추출 : get_page_product(bs_obj)
    - bs_obj를 전달받아서
    - 전체 제품정보를 추출
    - get_proudct_info(box)를 호출(제품 1개에 대한 정보가 반환)
    - df을 생성 후 저장
- 각 제품의 정보 추출 : get_proudct_info(box)
    - 제품정보 1개에 대해 제품명/가격/세일가격을 추출해서 반환

In [10]:
prd_dict = {'품목' : [],
            '가격' : [],
            '세일가격' : []}
df_fin = pd.DataFrame(prd_dict)

In [11]:
df_fin

,품목,가격,세일가격


In [12]:
# 1.  서버 요청 및 파싱객체 생성
def get_request_product(url) :
#     url = 'http://jolse.com/category/toners-mists/1019/'
    html = urlopen(url)
    htmls = html.read()
    
    bs_obj = BeautifulSoup(htmls,'html.parser')
    return bs_obj

In [13]:
# 2. 전체 제품 정보 크롤링(태그상태임) - 제품 1개 정보 크롤링 함수 호출 - df로 구성
# 시작 함수
def get_page_product(url) :
    global df_fin
    # 요청후 bs4 객체반환
    bs_obj = get_request_product(url)
    # 페이지내 전체 제품정보 추출
    boxes = bs_obj.findAll('div',{'class':'description'})
    
    # 각제품에 대한 정보추출 후 df 생성
    for box in boxes :
        res = pd.DataFrame(get_product_info(box),index=range(1,2))
        df_fin = pd.concat([df_fin,res], axis=0, ignore_index=True)

In [14]:
# 3. 제품 1개의 제품명/가격/세일가격 추출 후 dict 구성 후 반환
def get_product_info(box) :
    strong_tag = box.find('strong',{'class':'name'})
    prd_name = strong_tag.text.split(':')[1]
    # 가격
    price = box.find('ul').findAll('span')[1]
    sale_price = box.find('ul').findAll('span')[-1].text
    return{'품목':prd_name,'가격':price, '세일가격':sale_price}

In [15]:
url

'http://jolse.com/category/toners-mists/1019/'

In [16]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
get_page_product(url)

In [17]:
df_fin

,품목,가격,세일가격
0,[TIME DEAL] Haruharu WONDER Black Rice Hyalur...,USD 37.00,USD 16.59
1,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 18.70
2,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 31.45
3,Cos De BAHA Centella Facial Toner 200ml,USD 16.45,USD 13.98
4,Isntree Hyaluronic Acid Toner 200ml (Renewal),USD 19.30,USD 16.40
5,SOME BY MI AHA BHA PHA 30 Days Miracle Toner ...,USD 24.00,USD 20.40
6,SOME BY MI Galactomyces Pure Vitamin C Glow T...,USD 34.00,USD 28.90
7,ROUND LAB 1025 Dokdo Toner 200ml,USD 17.00,USD 14.45
8,SOME BY MI Snail Truecica Miracle Repair Tone...,USD 24.00,USD 20.40
9,COSRX AHA 7 WHITEHEAD POWER LIQUID 100ml,USD 17.81,USD 15.14


## 여러 페이지 추출
- 해당 쇼핑몰의 url 분석
- 각 페이지에 대하여 url에 파라미터가 다르게 전송됨
- https://jolse.com/category/toners-mists/1019?page= + 페이지번호
- url 작성시 페이지번호를 따로 부착

In [18]:

ssl._create_default_https_context = ssl._create_unverified_context
base_url ='https://jolse.com/category/toners-mists/1019?page='
for i in range(2,5) :
    url = base_url + str(i)
#     print(url)
    get_page_product(url)

In [19]:
df_fin

,품목,가격,세일가격
0,[TIME DEAL] Haruharu WONDER Black Rice Hyalur...,USD 37.00,USD 16.59
1,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 18.70
2,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 31.45
3,Cos De BAHA Centella Facial Toner 200ml,USD 16.45,USD 13.98
4,Isntree Hyaluronic Acid Toner 200ml (Renewal),USD 19.30,USD 16.40
...,...,...,...
91,YUNJAC WHOLE PLANT EFFECT ESSENTIAL TONER 150ml,USD 60.00,USD 57.00
92,celimax Jiwoogae Heartleaf BHA Peeling Pad 60...,USD 17.50,USD 14.87
93,Celimax JI WOO GAE Cica BHA Blamish Toner Pad...,USD 25.00,USD 21.25
94,FRUDIA Avocado Relief Essence Toner 195g,USD 24.21,USD 23.00


- best 상품에 대해서 첫번째 페이지에서만 추출되고 그 후 페이지에서는 추출되지 않도록 함수 수정
- 전체 페이지가 몇페이지인지를 크롤링을 통해서 얻어오고 해당 페이지 수 만큼 크롤링이 되게 코드 수정

In [20]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import ssl

In [21]:
prd_dict = {'품목':[],
            '가격':[],
            '세일가격':[]}
df_fin = pd.DataFrame(prd_dict)

In [22]:
# 요청 및 파싱 객체 생성
def get_request_product(url) :    
    htmls = urlopen(url)
    htmls = htmls.read()
    # 2. 파싱객체 생성
    bs_obj = BeautifulSoup(htmls, 'html.parser')
    return bs_obj

In [23]:
# 한개의 제품에 대한 정보 추출
def get_product_info(box) :
    strong_tag = box.find('strong',{'class':'name'})
    prd_name = strong_tag.text.split(':')[1]
    #가격
    price = box.find('ul').findAll('span')[1].text
    sale_price = box.find('ul').findAll('span')[-1].text
    return {'품목':prd_name, '가격':price, '세일가격':sale_price}

In [24]:
# 초기 호출 함수
def get_page_product(url) :
    global df_fin
    # 요청 및 파싱 객체 생성
    bs_obj= get_request_product(url) ## 요청 및 파싱 객체 생성
   # 페이지내 전체 제품정보 추출
    boxes = bs_obj.findAll('div',{'class':'description'})
    
    # 1페이지가 아니면 추천 제품은 다시 크롤링 하지 않게  조건식 생성
    #https://jolse.com/category/toners-mists/1019/?page=4
    # 베스트 제품이 4개로 고정시켜 놓은 것임
    # 베스트 제품 블럭과 제품 블럭이 구분되어 있으므로 그 내용을 반영하는게 효율적
    # 연습문제로 해결
    if url.split('=')[1] != '1' :
        boxes = boxes[4:len(boxes)]
       
    # 각 제품에 대한 정보 추출
    for box in boxes : 
        res = pd.DataFrame(get_product_info(box),index=range(1,2))
        df_fin = pd.concat([df_fin,res], axis=0, ignore_index=True)

### 마지막 페이지 번호 추출

In [26]:
import ssl
context = ssl._create_unverified_context()
url = 'https://jolse.com/category/toners-mists/1019/'
html = urlopen(url, context=context)
htmls = html.read()

In [27]:
# 2. 파싱객체 생성
bs_obj = BeautifulSoup(htmls,'html.parser')

In [28]:
# 마지막 페이지 번호
last_page = bs_obj.find('a',{'class':'last'})['href'].split('=')[1]
last_page

'26'

In [29]:
base_url = 'https://jolse.com/category/toners-mists/1019?page='
for i in range(1,int(last_page)+1) :
    url = base_url + str(i)
    get_page_product(url)

In [30]:
df_fin.tail()

,품목,가격,세일가격
503,O HUI Miracle Moisture Skin Softener Moist 150ml,USD 48.00,USD 40.80
504,O HUI Age Recovery Skin Softener 150ml,USD 47.98,USD 40.78
505,ZYMOGEN AC Dr. A-Sol 30ml,USD 13.98,USD 11.88
506,It's skin Aloe Relaxing Toner 150ml,USD 19.00,USD 16.15
507,It's skin Hyaluronic Acid Moisture Toner 150ml,USD 19.00,USD 16.15


In [31]:
'USD 16.45'[4]

'1'